In [1]:
import pyzx as zx
import random
from qiskit import QuantumCircuit
from qiskit.circuit import QuantumCircuit, ParameterVector
import csv
import time

In [2]:
def generate_H_S_CNOT_T_circuit(qubits, gates, p_t=0, seed=1000):
    random.seed(seed)  
    p_s = 0.333 * (1.0 - p_t)  
    p_had = 0.333 * (1.0 - p_t)  
    p_cnot = 0.333 * (1.0 - p_t)  

    c = zx.Circuit(qubits) 
    for _ in range(gates):
        r = random.random() 
        if r < p_had:
            c.add_gate("HAD", random.randrange(qubits))
        elif r < p_had + p_s:
            c.add_gate("S", random.randrange(qubits))
        elif r < p_had + p_s + p_t:
            c.add_gate("T", random.randrange(qubits))
        else:
            tgt = random.randrange(qubits)
            while True:
                ctrl = random.randrange(qubits)
                if ctrl != tgt:
                    break
            c.add_gate("CNOT", tgt, ctrl)
    return c

In [5]:
# seed固定
seed = 1002
random.seed(seed)

# パラメータ設定
qubits_list = [4, 8, 12, 16]
p_t_lists = [i * 0.04 for i in range(0, 11)]
gate_num_lists = [100, 500, 1000, 2000, 5000, 10000, 50000]

# CSV書き込みの準備
output_file = "../kekka1215/1215basic_50000_seed1002.csv"
fieldnames = [
    "optimize_name","data_name","gates","depth","t","H","otherRZ","CNOT","CZ","time"
]
start_all = time.time()
with open(output_file, "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()


    for qubit in qubits_list:
        for gate_num in gate_num_lists:
            for p_t in p_t_lists:
                try:
                    start = time.time()
                    # 回路作成
                    c = generate_H_S_CNOT_T_circuit(qubits = qubit,gates= gate_num,p_t = p_t,seed = seed)
                    c = zx.optimize.basic_optimization(c)

                    end = time.time()

                    #qiskit変換とカウント
                    circuit_qiskit = QuantumCircuit.from_qasm_str(c.to_basic_gates().to_qasm())
                    depth = circuit_qiskit.depth()
                    zx_dict = c.to_basic_gates().stats_dict()
                    qiskit_dict = circuit_qiskit.count_ops()

                    optimize_name = "basic"
                    dataname = "q" + str(qubit) + "_gates" + str(gate_num) + "_pt" +str(p_t)+ "_seeds" + str(seed)
                    gates_aft = zx_dict["gates"]


                    # 結果をCSVに書き込む
                    writer.writerow({
                        "optimize_name": optimize_name,
                        "data_name": dataname,
                        "gates": gates_aft,
                        "depth": depth,
                        "t": zx_dict["tcount"],
                        "H": qiskit_dict.get("h", 0),
                        "otherRZ": gates_aft - qiskit_dict.get("h", 0) - qiskit_dict.get("cx", 0) - qiskit_dict.get("cz", 0) - zx_dict["tcount"],
                        "CNOT": qiskit_dict.get("cx", 0),
                        "CZ": qiskit_dict.get("cz", 0),
                        "time": end - start
                    })

                    print(f"done. Qubits={qubit}, Gate_Num={gate_num}, P_T={p_t},seed={seed},time={end-start}")

                except Exception as e:
                    print(f"Error for Qubits={qubit}, Gate_Num={gate_num}, P_T={p_t}: {e}")

end_all = time.time()
print(f"Total time: {end_all - start_all}")

done. Qubits=4, Gate_Num=100, P_T=0.0,seed=1002,time=0.004678964614868164
done. Qubits=4, Gate_Num=100, P_T=0.04,seed=1002,time=0.0023813247680664062
done. Qubits=4, Gate_Num=100, P_T=0.08,seed=1002,time=0.0021500587463378906
done. Qubits=4, Gate_Num=100, P_T=0.12,seed=1002,time=0.0017638206481933594
done. Qubits=4, Gate_Num=100, P_T=0.16,seed=1002,time=0.0018303394317626953
done. Qubits=4, Gate_Num=100, P_T=0.2,seed=1002,time=0.0025331974029541016
done. Qubits=4, Gate_Num=100, P_T=0.24,seed=1002,time=0.0018608570098876953
done. Qubits=4, Gate_Num=100, P_T=0.28,seed=1002,time=0.002450704574584961
done. Qubits=4, Gate_Num=100, P_T=0.32,seed=1002,time=0.0028846263885498047
done. Qubits=4, Gate_Num=100, P_T=0.36,seed=1002,time=0.0023126602172851562
done. Qubits=4, Gate_Num=100, P_T=0.4,seed=1002,time=0.0020771026611328125
done. Qubits=4, Gate_Num=500, P_T=0.0,seed=1002,time=0.013270378112792969
done. Qubits=4, Gate_Num=500, P_T=0.04,seed=1002,time=0.015858173370361328
done. Qubits=4, Gate